In [ ]:
%pip -q install -U graphrag azure-ai-documentintelligence langchain langchain-community langchain-openai

In [ ]:
import os
from dotenv import load_dotenv

# notebook 환경변수를 위해 parent directory의 .env 파일을 로드합니다.
load_dotenv(override=True)

In [ ]:
import os
from langchain_community.document_loaders.doc_intelligence import AzureAIDocumentIntelligenceLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores.azuresearch import AzureSearch

loader = AzureAIDocumentIntelligenceLoader(
    api_endpoint=os.environ["AZURE_DOCUMENTINTELLIGENCE_ENDPOINT"],
    api_key=os.environ["AZURE_DOCUMENTINTELLIGENCE_API_KEY"],
    file_path="./resources/대한민국 헌법.pdf",
    api_model="prebuilt-read",
)
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chunks = splitter.split_documents(docs)

In [ ]:
from pathlib import Path

Path("input").mkdir(parents=True, exist_ok=True)
with open("input/chunks.txt", "w", encoding="utf-8") as f:
    for chunk in chunks:
        f.write(chunk.page_content + "\n")

In [ ]:
import pandas as pd
from pathlib import Path
import graphrag.api as api
from graphrag.config.load_config import load_config
from graphrag.index.typing.pipeline_run_result import PipelineRunResult

root_dir = Path("./")

# 1) settings.yaml + .env 로부터 GraphRAG 설정 로드
config = load_config(root_dir)

# 2) 인덱싱 파이프라인 실행
index_result: list[PipelineRunResult] = await api.build_index(config=config)

# 3) 워크플로우별 성공/실패 출력
for workflow_result in index_result:
    status = "success" if not workflow_result.errors else f"error\n{workflow_result.errors}"
    print(f"Workflow: {workflow_result.workflow}\tStatus: {status}")

# 4) 인덱싱 결과인 parquet 파일 로드 (q&a에서 사용)
entities = pd.read_parquet(root_dir / "output" / "entities.parquet")
communities = pd.read_parquet(root_dir / "output" / "communities.parquet")
community_reports = pd.read_parquet(root_dir / "output" / "community_reports.parquet")

print("entities.head():")
print(entities.head())

In [29]:
import pandas as pd
from pathlib import Path

import graphrag.api as api
from graphrag.config.load_config import load_config

PROJECT_DIR = Path("./")
config = load_config(PROJECT_DIR)

# 인덱싱 후 생성된 parquet 로드
entities = pd.read_parquet(PROJECT_DIR / "output" / "entities.parquet")
communities = pd.read_parquet(PROJECT_DIR / "output" / "communities.parquet")
community_reports = pd.read_parquet(PROJECT_DIR / "output" / "community_reports.parquet")

question = "대통령은 누구이고, 어떤 책임과 의무를 가져 ?"

# ---- Global Search ----
response, context = await api.global_search(
    config=config,
    entities=entities,
    communities=communities,
    community_reports=community_reports,
    community_level=2,                # 커뮤니티 계층 (보통 1~3)
    dynamic_community_selection=False, # true 로 하면 질문에 맞게 커뮤니티 자동 선택
    response_type="Multiple Paragraphs",
    query=question,
)
print("=== Answer ===")
print(response)
print("\n=== Context (debug) ===")
print(context)


=== Answer ===
# 대한민국 대통령의 정의와 역할

대한민국 대통령은 국가의 최고 행정권자이자 정부의 수반이며, 국가의 원수로서 국내외적으로 대한민국을 대표한다. 대통령은 국민에 의해 직접 선출되며, 단임 5년의 임기를 가진다. 대통령은 국가의 독립을 수호하고, 평화적 통일을 추구하며, 헌법과 법률에 따라 국가 운영의 중심적 역할을 수행한다 [Data: Reports (1, 15, 19)].

# 대통령의 주요 권한

대통령은 행정부의 수장으로서 정부를 대표하고, 국가의 주요 정책을 수립·집행한다. 대통령의 권한에는 다음과 같은 사항이 포함된다:

- **법률 공포 및 주요 공직자 임명**: 대통령은 국무총리, 국무위원, 법관, 군 지휘관 등 주요 공직자를 임명하며, 법률을 공포한다 [Data: Reports (1, 8, 14, 17, 41)].
- **국무회의 주재**: 대통령은 국무회의(국가 최고 정책 심의기구)를 주재하여 주요 정책, 인사, 예산, 조약, 법률안 등을 심의·결정한다 [Data: Reports (1, 41)].
- **국가 비상사태 시 권한 행사**: 대통령은 국가 비상시 긴급명령 발동, 계엄 선포, 사면·감형·복권·훈장 수여 등의 권한을 가진다. 이러한 권한은 국무회의의 심의와 법적 절차를 거쳐 행사된다 [Data: Reports (1, 2, 28)].
- **국군 통수권**: 대통령은 국군의 최고 통수권자로서 군사 작전과 국가 방위 전략을 지휘하며, 군의 정치적 중립을 보장한다 [Data: Reports (1, 15)].
- **헌법 개정안 및 법률안 제출**: 대통령은 헌법 개정안, 법률안, 예산안 등을 국회에 제출할 권한이 있다 [Data: Reports (8, 16, 43)].
- **국제적 대표 및 외교**: 대통령은 국가를 대표하여 외교 정책을 수립·집행하고, 국제적 의무를 이행한다 [Data: Reports (2, 17, 41)].

# 대통령의 책임과 의무

대통령의 책임과 의무는 헌법과 법률에 따라 엄격히 규정된

In [ ]:
text_units = pd.read_parquet(root_dir / "output" / "text_units.parquet")
relationships = pd.read_parquet(root_dir / "output" / "relationships.parquet")

# covariates 는 없어도 됨 → 없으면 None 으로
try:
    covariates = pd.read_parquet(root_dir / "output" / "covariates.parquet")
except FileNotFoundError:
    covariates = None
    
# ---- Local Search ----
response, context = await api.local_search(
    config=config,
    entities=entities,
    communities=communities,
    community_reports=community_reports,
    text_units=text_units,
    relationships=relationships,
    covariates=covariates,
    community_level=2,
    response_type="Multiple Paragraphs",
    query=question,
)
print("=== Answer ===")
print(response)
print("\n=== Context (debug) ===")
print(context)

=== Answer ===
## 대한민국 대통령의 정의

대한민국 대통령(대통령)은 국가의 원수이자 행정부의 수반으로, 국내외적으로 대한민국을 대표하는 최고 권력자입니다. 대통령은 국민의 직접 선거로 선출되며, 임기는 5년이고 연임은 불가능합니다. 대통령은 국가의 독립, 영토의 보전, 국가의 계속성과 헌법 수호, 그리고 평화적 통일을 위한 성실한 의무를 집니다 [Data: Entities (121); Sources (9, 10)].

## 대통령의 주요 책임과 의무

### 1. 헌법적·국가적 책무
- **헌법 준수 및 국가 보위**: 대통령은 취임 시 헌법을 준수하고 국가를 보위하며, 국민의 자유와 복리 증진, 민족문화 창달에 노력할 것을 선서합니다 [Data: Entities (185); Sources (9)].
- **국가의 독립과 영토 보전**: 국가의 독립과 영토의 보전을 책임지며, 국가의 계속성과 헌법 수호의 책무를 집니다 [Data: Entities (121); Sources (9)].
- **평화적 통일**: 조국의 평화적 통일을 위한 성실한 의무를 집니다 [Data: Entities (121); Sources (9)].

### 2. 행정권 및 임명권
- **행정권 행사**: 대통령을 수반으로 하는 정부에 행정권이 속합니다 [Data: Entities (28, 121); Sources (9)].
- **공무원 임면권**: 대통령은 헌법과 법률에 따라 공무원을 임명하거나 해임할 수 있습니다 [Data: Entities (121); Sources (10)].
- **주요 공직자 임명**: 국무총리, 국무위원, 행정각부의 장, 헌법재판소 재판관, 대법원장, 검찰총장, 국립대학교총장 등 주요 공직자를 임명합니다 [Data: Entities (121, 143, 228, 204, 207); Sources (10)].

### 3. 입법 및 예산 관련 권한
- **법률안 제출 및 공포**: 정부의 법률안과 예산안을 국회에 제출하고, 국회에서 의결된 법률안을

In [28]:
# ---- Basic Search ----
response, context = await api.basic_search(
    config=config,
    query=question,
    text_units=text_units,
)
print("=== Answer ===")
print(response)
print("\n=== Context (debug) ===")
print(context)

=== Answer ===
## 대통령의 정의와 지위

대한민국 헌법에 따르면 대통령은 국가의 원수이며, 외국에 대하여 국가를 대표하는 최고 권력자입니다. 대통령은 국가의 독립, 영토의 보전, 국가의 계속성과 헌법을 수호할 책무를 지니며, 조국의 평화적 통일을 위한 성실한 의무도 집행합니다. 또한 행정권은 대통령을 수반으로 하는 정부에 속합니다 [Data: Sources (8, 9)].

## 대통령의 책임과 의무

대통령은 헌법과 법률에 따라 다양한 책임과 의무를 집행합니다. 주요 내용은 다음과 같습니다:

- **헌법 수호 및 국가 보위**: 대통령은 헌법을 준수하고 국가를 보위하며, 국민의 자유와 복리 증진, 민족문화의 창달에 노력할 것을 국민 앞에 선서합니다 [Data: Sources (9)].
- **국가의 대표 및 외교권 행사**: 대통령은 국가를 대표하여 조약을 체결·비준하고, 외교사절을 신임·접수·파견하며, 선전포고와 강화 등 외교적 권한을 행사합니다 [Data: Sources (9)].
- **국군 통수권**: 대통령은 국군을 통수하며, 군사에 관한 중요 사항을 결정합니다 [Data: Sources (9, 10)].
- **입법 및 명령권**: 대통령은 법률에서 구체적으로 범위를 정하여 위임받은 사항과 법률 집행에 필요한 사항에 관해 대통령령을 발할 수 있습니다. 긴급한 국가 위기 상황에서는 국회의 집회를 기다릴 여유가 없을 때 한하여 긴급명령을 발할 수 있습니다 [Data: Sources (9, 10)].
- **공무원 임면권**: 대통령은 헌법과 법률이 정하는 바에 따라 공무원을 임면할 권한을 가집니다 [Data: Sources (10)].
- **사면권 및 영전 수여**: 대통령은 법률이 정하는 바에 따라 사면, 감형, 복권을 명할 수 있으며, 훈장 등 영전을 수여할 수 있습니다 [Data: Sources (10)].
- **국회와의 관계**: 대통령은 국회에 출석하여 발언하거나 서한으로 의견을 표시할 수 있습니다 [Data: Source